In [137]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
import seaborn as sns
import keras
import warnings
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
warnings.filterwarnings("ignore")
import keras
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings("ignore")
import keras_tuner as kt
import tensorflow as tf

In [138]:
data = pd.read_csv('../data/regional_multivariate_data.csv')
enc = pd.read_csv('../data/regional_datasets/multivariate/enc_data_multi.csv').drop(columns = 'Unnamed: 0').set_index('date')
esc = pd.read_csv('../data/regional_datasets/multivariate/esc_data_multi.csv').drop(columns = 'Unnamed: 0').set_index('date')
midatl = pd.read_csv('../data/regional_datasets/multivariate/mid_atlantic_data_multi.csv').drop(columns = 'Unnamed: 0').set_index('date')
mount = pd.read_csv('../data/regional_datasets/multivariate/mountain_data_multi.csv').drop(columns = 'Unnamed: 0').set_index('date')
ne = pd.read_csv('../data/regional_datasets/multivariate/new_england_data_multi.csv').drop(columns = 'Unnamed: 0').set_index('date')
pac = pd.read_csv('../data/regional_datasets/multivariate/pacific_data_multi.csv').drop(columns = 'Unnamed: 0').set_index('date')
satl = pd.read_csv('../data/regional_datasets/multivariate/south_atlantic_data_multi.csv').drop(columns = 'Unnamed: 0').set_index('date')
wnc = pd.read_csv('../data/regional_datasets/multivariate/wnc_data_multi.csv').drop(columns = 'Unnamed: 0').set_index('date')
wsc = pd.read_csv('../data/regional_datasets/multivariate/wsc_data_multi.csv').drop(columns = 'Unnamed: 0').set_index('date')

In [168]:
def process_normalize(df):
    x = df.drop(columns = ['new_confirmed', 'region'] + [col for col in df.columns if col[0].isdigit()])
    lag1df = x.shift(1)
    lag1df.columns = ['lag1_' + str(col) for col in lag1df.columns]
    df = df[['new_confirmed'] + [col for col in df.columns if col[0].isdigit()]].join(lag1df)
    df = df.dropna()
    normalizer = Normalizer()
    norm_df = normalizer.fit_transform(df.drop(columns = 'new_confirmed'))
    res = pd.concat([df.new_confirmed, 
                     pd.DataFrame(norm_df, columns = df.columns.drop('new_confirmed'), index = df.index)], axis = 1)
    return res

In [169]:
def make_series(data, length):
    series=[]
    for i in range(len(data)-length):
        subdf=data[i:i+length]
        series.append((subdf.index[-1], subdf['new_confirmed'].values[-1], subdf.drop('new_confirmed', axis=1).values))
    return series

In [170]:
series = make_series(process_normalize(enc), 7)
print(series[0][0],series[0][1].shape,series[0][2].shape)
size = round(len(series)*0.2)
train = series[:-size]
test = series[-size:]
val = train[-round(len(train)*0.15):]
train = train[:-round(len(train)*0.15)]
print(train[0][0],train[0][1].shape,train[0][2].shape)
print(val[0][0],val[0][1].shape,val[0][2].shape)
learning_rate = 0.01
batch_size = 32
epochs = 500
sequence_length = 7
x=np.array([x[2].reshape(7,51) for x in train])
y=np.array([x[1] for x in train])
xval=np.array([x[2].reshape(7,51) for x in val])
yval=np.array([x[1] for x in val])
model = keras.Sequential()
model.add(keras.layers.LSTM(units = 32,
                                return_sequences = True,
                                input_shape = (x.shape[1], x.shape[2])))
model.add(keras.layers.Dropout(rate= 0.2))
# model.add(keras.layers.LSTM(units = 128))
# model.add(keras.layers.Dropout(rate= 0.2))
model.add(keras.layers.Dense(1, activation = 'relu'))
model.compile(optimizer=keras.optimizers.legacy.Adam(learning_rate=learning_rate), loss="mae")
print(x.shape)
print(y.shape)
history = model.fit(
    x=x,y=y,
    epochs=epochs,
    batch_size= batch_size,
    shuffle=True,
    validation_data = (xval, yval),
    verbose = 0
)
xtest=np.array([x[2].reshape(7,51) for x in test])
ytest=np.array([x[1] for x in test])
test_pred = model.predict(xtest)

2020-03-18 () (7, 51)
2020-03-18 () (7, 51)
2021-10-16 () (7, 51)
(577, 7, 51)
(577,)
6/6 [==============================] - 0s 879us/step


In [175]:
x

array([[[ 2.92511283e-03,  1.92017859e-03,  2.46850985e-03, ...,
          2.82751394e-04, -2.81413262e-05,  1.68927249e-04],
        [ 7.13301468e-03,  4.62662707e-03,  5.13076494e-03, ...,
          6.99886395e-04,  8.06461142e-06,  4.11851658e-04],
        [ 1.15631206e-02,  6.84708233e-03,  8.38546915e-03, ...,
          1.00254904e-03, -4.11917783e-05,  4.35619414e-04],
        ...,
        [ 1.59854590e-02,  1.20253949e-02,  1.18273916e-02, ...,
          1.14078889e-03, -2.21079145e-04,  2.36992715e-04],
        [ 2.26494850e-02,  1.43806254e-02,  1.51874454e-02, ...,
          1.51195555e-03, -1.82080264e-04,  2.53936955e-04],
        [ 3.86507866e-02,  2.70053548e-02,  3.39951237e-02, ...,
          2.60269127e-03, -2.35223241e-04,  4.45313677e-04]],

       [[ 7.13301468e-03,  4.62662707e-03,  5.13076494e-03, ...,
          6.99886395e-04,  8.06461142e-06,  4.11851658e-04],
        [ 1.15631206e-02,  6.84708233e-03,  8.38546915e-03, ...,
          1.00254904e-03, -4.11917783e

In [156]:
df = enc
x = df.drop(columns = ['new_confirmed', 'region'] + [col for col in df.columns if col[0].isdigit()])
lag1df = x.shift(1)
lag1df.columns = ['lag1_' + str(col) for col in lag1df.columns]
df = df[['new_confirmed'] + [col for col in df.columns if col[0].isdigit()]].join(lag1df)
df = df.dropna()
normalizer = Normalizer()
norm_df = normalizer.fit_transform(df)
data = pd.DataFrame(norm_df, columns = df.columns, index = df.index)
series=[]
length = 7
for i in range(len(data)-length):
    subdf=data[i:i+length]
    series.append((subdf.index[-1], subdf['new_confirmed'].values[-1], subdf.drop('new_confirmed', axis=1).values))

In [151]:
data

array([[ 4.06658665e-03,  2.92508864e-03,  1.92016272e-03, ...,
         2.82749056e-04, -2.81410935e-05,  1.68925852e-04],
       [ 9.83557605e-03,  7.13266965e-03,  4.62640327e-03, ...,
         6.99852541e-04,  8.06422134e-06,  4.11831736e-04],
       [ 1.12975953e-02,  1.15623827e-02,  6.84664535e-03, ...,
         1.00248506e-03, -4.11891494e-05,  4.35591613e-04],
       ...,
       [ 1.69925383e-04,  1.62713353e-04,  1.24499081e-04, ...,
         2.78391729e-07,  1.44600386e-07,  2.81390079e-07],
       [ 2.93732249e-04,  1.69922031e-04,  1.31414715e-04, ...,
         2.78386237e-07,  1.34800203e-07,  2.87840183e-07],
       [ 9.56960512e-05,  2.93125527e-04,  1.28718854e-04, ...,
         2.77811214e-07,  8.96811706e-08,  2.84280122e-07]])